# Required Packages

In [ ]:
# !pip install plotly
# !pip install pandas
# !pip install numpy
# !pip install dash
# !pip install dash-daq

# Packages

In [ ]:
from plotly import graph_objs as go

import dash

import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output

import pandas as pd

# Initilize Dash App

In [ ]:
app = dash.Dash()

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

**Data which I've used contains start time and end time along with a distance covered by a person by any mean of transport.**

In [ ]:
path = '../data.xlsx'
data = pd.read_excel(path)

# Scatter Plot

In [ ]:
app.layout = html.Div([
    html.H1('Scatter Plot'),
    dcc.Graph(
        id = 'scatter',
        figure = {
            'data' : [
                go.Scatter(
                    x = [1,2,4,5,5,2,2,16],
                    y = [2,5,2,1,3,5,1,0],
                    mode = 'markers'
                )
            ], #data
            'layout' : go.Layout(
                title = 'Scatter Plot',
                xaxis = {'title' : 'Distance'},
                yaxis = {'title' : 'Duration in Seconds'},
                hovermode = 'closest'
            ) #layout
        } # figure
    ) # Graph
])

# Bar Chart

In [ ]:
app.layout = html.Div([
    html.H1('Unlimit Insights'),
    html.Div('Data Product Dashboard'),
    dcc.Graph(
        id = 'ex1',
        figure = {
            'data' : [
                        {'x' : [1,2,4,5,6],'y':[2,4,8,3,1],'type':'bar','name':'cars'},
                        {'x' : [4,9,1,2],'y':[1,4,5,3],'type':'bar','name':'bike'},
                        {'x' : [1,2,8,9,6],'y':[3,5,7,1,10],'type':'lie','name':'road'}
                    ],
            'layout' : {'title' : 'Sample Bar'}
        }
    )
])

# Drop Down

In [ ]:
x = {
    'a' : 'category',
    'b' : 'category',
    'c' : 'category',
    'd' : 'numeric',
    'e' : 'numeric',
    'f' : 'timestamp'
}
options = [ {'label' : str(k).title(), 'value' : str(k)} for k,v in x.items() if v == 'category']
options

In [ ]:
app.layout = html.Div([
    html.H1('Drop Down'),
    dcc.Dropdown(
        id='drop_down',
        options = options,
        placeholder = 'Select Columns'
    )
])

# Generate Table

In [ ]:
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )

app.layout = html.Div(children=[
    html.H4(children='UBI'),
    generate_table(data)
])


# Combine Charts

In [ ]:
app.layout = html.Div([
    html.H1('Drop Down'),
    dcc.Dropdown(
        id='drop_down',
        options = options,
        placeholder = 'Select Columns'
    ),
    html.Label("Slider"),
    dcc.Slider(
        min = 1,
        max = 10,
        steps = 0.5,
        marks = {i:i for i in range(10)}
    )
])

# Range Slider

In [ ]:
app.layout = html.Div([
    html.Label("Range Slider"),
    dcc.RangeSlider(
        min = 1,
        max = 10,
        value = [5,10],
        marks = {i:i for i in range(10)}
    )
])

# I/p O/p Components

In [ ]:
app.layout = html.Div([
    html.Label("Text"),
    dcc.Input(
        placeholder = 'Enter Name',
        type='text'
    ),
    html.Div([
        html.Label('Numeric'),
        dcc.Input(
            placeholder = 'Select Number',
            value = '0',
            type = 'number'
        )
    ])
])

# Heatmap

In [ ]:
corr = data.corr()
app.layout = html.Div([
        dcc.Graph(
            id = 'dist_plot',
            figure = {
                'data' :[
                    go.Heatmap(
                        z = list(corr.values.tolist()),
                        x = corr.columns.tolist(),
                        y = corr.index.tolist()
                    )
                ],
                'layout' : {
                    'margin' : {'b' : 100, 't' : 100, 'l' : 100, 'r' : 100},
                    'legend' : {'x' : 0}
                }
            }
        ) # Histogram Graph
])

# Timeseries Plot

## Basic

In [ ]:
app.layout = html.Div([
    dcc.Graph(
        id = 'ts_plot',
        figure = {
            'data' : [
                go.Scatter(x=data.start_time,y=data.distance)
            ]
        }
    )
])# div

## With Range Slider

In [ ]:
data.start_time.max().month

In [ ]:
app.layout = html.Div([
    dcc.Graph(
        id = 'ts_plot',
        figure = {
            'data' : [
                go.Scatter(x=data.start_time,y=data.distance)
            ]
        }
    ), # Graph
    dcc.RangeSlider(
        id = 'range_month',
        min = data.start_time.min().month,
        max = data.start_time.max().month,
        marks = {i:i for i in range(12)}
    ) # Range Slider
])# div

**Updating the timeseries plot using the range slider**

In [ ]:
@app.callback(
    Output('ts_plot','figure'),
    [Input('range_month','value')]
)
def update_graph(value):
    # print(value) -> Returns list with 2 values
    if value[0] < value[1]:
        filterData = data.loc[data['start_time'].apply(lambda x  : x.month in range(value[0],value[1])),
                              ['start_time','distance']]
    else:
        filterData = data.loc[data['start_time'].apply(lambda x  : x.month in range(value[0],value[1],-1)),
                              ['start_time','distance']]
    return {
        'data' : [
            go.Scatter(x = filterData['start_time'], y = filterData['distance'])
        ]
    }

# Run App

In [ ]:
if __name__ == '__main__':
    app.run_server(port=4050)